<a href="https://colab.research.google.com/github/sunxueliang96/WF-FrameWork/blob/master/RNN_PADDING_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Mount Google Drive as folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/'My Drive'/datasets/no_paded/close_world/walkiebatch

/content/drive/My Drive/datasets/no_paded/close_world/walkiebatch


In [5]:
import numpy as np
import pickle
print('loading data...')
with open('X_walkiebatch.pkl','rb') as handle:
  X = np.array(pickle.load(handle))
with open('y_walkiebatch.pkl','rb') as handle:
  y = np.array(pickle.load(handle))
print('the shape of X',X.shape)
print('the shape of y',y.shape)


loading data...
the shape of X (11868,)
the shape of y (11868,)


In [0]:
from keras import Sequential, Model
from keras.preprocessing import sequence
from keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Dropout, Masking
from keras.layers.core import Activation, Flatten
def LSTM_DNN(maxlen,NB_CLASSES):
    latent_dim = 256
    NB_repeat = 3

    model = Sequential() 
    model.add(Masking(mask_value=0, input_shape=(maxlen,1)))
    model.add(LSTM(75))
    model.add(RepeatVector(NB_repeat))
    model.add(LSTM(50, return_sequences=True))
    model.add(TimeDistributed(Dense(maxlen, activation= 'softmax' ))) 
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    model.add(Dense(NB_CLASSES))
    model.add(Activation('softmax'))
    return model
def run_LSTM_DNN(X_train,y_train,X_test,y_test):
 
    print('Pad sequences to ',maxlen)
    x_train = X_train[:]
    x_test = X_test[:]
    x_train = sequence.pad_sequences(x_train, maxlen=maxlen,padding='post',truncating='post')
    x_test = sequence.pad_sequences(x_test, maxlen=maxlen,padding='post',truncating='post')
    x_train = x_train[:,:,np.newaxis]
    x_test = x_test[:,:,np.newaxis]
    y_train = np_utils.to_categorical(y_train, NB_CLASSES)
    y_test = np_utils.to_categorical(y_test, NB_CLASSES)
    print('the shape of x_train',x_train.shape)
    print('the shape of y_train',y_train.shape)
    print('the shape of x_test',x_test.shape)
    print('the shape of y_test',y_test.shape)

    model_LSTM_DNN = LSTM_DNN(maxlen,NB_CLASSES)
    model_LSTM_DNN.summary()
    model_LSTM_DNN.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])#, precision, recall, fmeasure])
    history = model_LSTM_DNN.fit(x_train,y_train,batch_size=BATCH_SIZE,epochs=NB_EPOCH,validation_data=(x_test,y_test),verbose=1)
    score = model_LSTM_DNN.evaluate(x_test,y_test,verbose=VERBOSE)
    #print(score)
    return(history.history)

In [0]:
from sklearn.model_selection import train_test_split
from collections import Counter
from keras.utils import np_utils
import numpy as np

print('Average sequence length: {}'.format(np.mean(list(map(len, X)), dtype=int)))
maxlen = 5000
NB_CLASSES = len(Counter(y).keys())
BATCH_SIZE = 63
NB_EPOCH = 200
print('number of classes is {}'.format(NB_CLASSES))


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

run_LSTM_DNN(X_train,y_train,X_test,y_test)

Average sequence length: 4471
number of classes is 108
Pad sequences to  5000
the shape of x_train (8307, 5000, 1)
the shape of y_train (8307, 108)
the shape of x_test (3561, 5000, 1)
the shape of y_test (3561, 108)
Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_4 (Masking)          (None, 5000, 1)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 75)                23100     
_________________________________________________________________
repeat_vector_13 (RepeatVect (None, 3, 75)             0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 3, 50)             25200     
_________________________________________________________________
time_distributed_12 (TimeDis (None, 3, 5000)           255000    
___________________________________

In [36]:
x_train.reshape(x_train.shape[0],None,1)

TypeError: ignored